<a href="https://colab.research.google.com/github/iamjaco/nlp_paraphrase/blob/main/NLP_Paraphraser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 3.4 MB 4.1 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 3.3 MB 39.3 MB/s 
     |████████████████████████████████| 895 kB 73.4 MB/s 
     |████████████████████████████████| 596 kB 73.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [2]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
import nltk
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [10]:
# helper functions
import re

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text


def capitalize(text):
    sentences = text.split('. ')
    sentences = [sentence[0].capitalize() + sentence[1:] for sentence in sentences]
    return '. '.join(sentences)


def to_sentences(text):
  text = [sentence for sentence in sent_tokenize(text) if sentence]
  return text


def Flesch_Kincaid(text): 
    def count_words(text):
        """
        Returns the number of words in a text.
        """
        return len(text.split())
    
    def count_sentences(text):
        """
        Returns the number of sentences in a text.
        """
        return len(sent_tokenize(text))
    
    def count_syllables(word, isName=True):
        """Returns the number of syllables in a word.
           Most syllables are defined as vowels, but some are defined as dipthongs or diphthongs.  We will follow the later definition since it is more inclusive and produces better results."""
    
        word = word.lower()  # convert to lower case before counting vowels in word 
    
         # count vowel groups using regular expression: any vowel followed by zero or more vowels or vowels with y between them  
    
        pattern = re.compile("[aeiouy]+")  # create pattern for one vowel group including all vowels and y's between them  
    
         # find all instances of this pattern in the word (matches)  
    
        matches = re.findall(pattern, word)  
    
         # add up total number of matching vowel groups (syllables)  
    
        numVowelGroups = 0  # initialize counter  
    
        for match in matches:  # loop through list of matches  
            numVowelGroups += 1  # increment counter  
    
        return numVowelGroups
        
    return 206.835 - 1.015 * (count_words(text) / count_sentences(text)) - 84.6 * (count_syllables(text) / count_words(text))


def best_sentence(text):
    max_score = 0
    best = ''

    for sentence in text:
        score = Flesch_Kincaid(sentence)

        if score > max_score:
            max_score = score
            best = sentence

    return best



def paraphrase(text):
  rewritten = []
  for s in text:
      r = get_response(s,10,10)
      rewritten.append(best_sentence(r))


  rewritten = ' '.join(rewritten)
  return rewritten       


In [7]:
from nltk import sent_tokenize
nltk.download('punkt')
input_text = """
The energy and excitement in machine learning and deep learning communities are infectious these days. So many groundbreaking advances are happening in this area but I have often found myself wondering why the only thing that makes it all shine — yes I am talking about the dark horse of deep learning the data is so underappreciated. The last few years of DL research have given me much joy and excitement and I carry hope now that going forward we can see some exciting progress in this space that explore advances in deep learning in conjunction with data! In this article, I summarise some of the recent developments in the deep learning space that I have been blown away by.
"""

input_text = to_sentences(input_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
paraphrase(input_text)

'The machine learning and deep learning communities are fun to be a part of. I wonder why the only thing that makes it all shine is the dark horse of deep learning the data. The last few years of deep learning research have given me a lot of joy and excitement and I hope that we can see some exciting progress in this area going forward. I have been blown away by the recent developments in the deep learning space.'